# Création du Dataset

Dans cette première partie, nous constituons un dataset avec les différents signes de l'alphabet. Nous avons choisi de collecter des images traitées en amont via un "filtre" (Thresholded) qui permettra de différencier au mieux chaque signe effectué avec la main.

Durant cette aquisition de donnée, nous chargeons préalablement le background, qui ne sera pas pris en compte lorsque nous positionnerons la main. Une fois fait, 300 images seront enregistrées dans un dossier conçu à cet effet. L'opération sera reproduite pour chaque catégorie (chaque signe de l'alphabet).

Le dataset constitué (près de 8100 images) sera compressé pour l'importer sur *Colab* afin d'entrainer notre modèle.

In [41]:
import cv2
import os
import time
import numpy as np

In [76]:
background = None
accumulated_weight = 0.9

# Création des dimensions du ROI

ROI_top = 50
ROI_bottom = 300
ROI_right = 50
ROI_left = 300

# Project: gesture-recognition, License: MIT License, OpenSource

def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [79]:
# Fonction pour l'application du filtre (Thresholded)
def segment_hand(frame, threshold=20):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    thresholded = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    
    return (thresholded)

In [111]:
# Ouverture de la caméra
cam = cv2.VideoCapture(0)

# Définition des variables (nombre de frame, dossier d'images)
num_frames = 0
element = '_SPACE'

num_imgs_taken = 0

while True:
    ret, frame = cam.read()
    
    # Eviter l'inversement de l'image
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    # Traitement
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    
    if num_frames < 100:
        cal_accum_avg(gray_frame, accumulated_weight)
        
        # Détection du Background
        if num_frames <= 299:
            cv2.putText(frame_copy, "Chargement..", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            
    # Configuration (Nous pouvons mettre notre main et la positionner correctement avant enregistrement)
    elif num_frames <= 300: 
        hand = segment_hand(gray_frame)
        cv2.putText(frame_copy, "Configuration.." + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)
        
        # Voir si la main est bien détectée
        if hand is not None:
            thresholded = hand
            
            # Dessiner les contours de la main
            cv2.putText(frame_copy, str(num_frames)+" Pour "+ str(element),(70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            
            # Affiche l'image prise
            cv2.imshow("Image traitée", thresholded)
    
    else: 
        # Filtre sur la main
        hand = segment_hand(gray_frame)
        
        # Repérer si la main est là
        if hand is not None:
            
            thresholded = hand
            
            # Détection de la main (avec le filtre)
            cv2.putText(frame_copy, str(num_frames), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.putText(frame_copy, str(num_imgs_taken) + 'Enregistrement pour ' + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
            # Enregistrement des images de la main traitée dans la frame Filtre
            cv2.imshow("Image traitée", thresholded)
            if num_imgs_taken <= 299:
                cv2.imwrite(r"C:\\Users\\utilisateur\\Documents\\dataset2\\"+str(element)+"\\"+str(num_imgs_taken+1) +'.jpg', thresholded)
                
            else:
                break
            num_imgs_taken +=1
            
        else:
            cv2.putText(frame_copy, 'Pas de main détectée !', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Dessine une ROI de la frame Filtre
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
    
    # Incrementation
    num_frames += 1
    # Montre la frame Filtre
    cv2.imshow("Filtre", frame_copy)
    # Ferme l'application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cam.release()